# Sample for using transformer with KFServing SDK 

The notebook shows how to use KFServing SDK to create InferenceService with transformer, predictor.

In [24]:
from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TransformerSpec
from kfserving import V1alpha2PyTorchSpec
from kfserving import V1alpha2CustomSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1Container
from kubernetes.client import V1ResourceRequirements
import kubernetes.client
import os
import requests 
import json
import numpy as np

## Define InferenceService with Transformer

Add predictor and transformer on the endpoint spec

In [19]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                            min_replicas=1,
                            pytorch=V1alpha2PyTorchSpec(
                              storage_uri='gs://kfserving-samples/models/pytorch/cifar10',
                              model_class_name= "Net",
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'1Gi'},
                                  limits={'cpu':'100m', 'memory':'1Gi'}))),
                          transformer=V1alpha2TransformerSpec(
                            min_replicas=1,
                            custom=V1alpha2CustomSpec(
                              container=V1Container(
                              image='yuzisun/image-transformer:latest',
                              name='user-container',
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'1Gi'},
                                  limits={'cpu':'100m', 'memory':'1Gi'})))))
    
isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='cifar10', namespace='kubeflow'),
                          spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))

## Create InferenceService with Transformer

Call KFServingClient to create InferenceService.

In [ ]:
KFServing = KFServingClient()
KFServing.create(isvc)

## Check the InferenceService

In [22]:
KFServing.get('cifar10', namespace='kubeflow', watch=True, timeout_seconds=120)

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
cifar10              Unknown                                    http://cifar10-predict.kubeflow.example.com       
cifar10              Unknown                                    http://cifar10-predict.kubeflow.example.com       
cifar10              True                                       http://cifar10-predict.kubeflow.example.com       


## Predict the image

In [25]:
api_instance = kubernetes.client.CoreV1Api(kubernetes.client.ApiClient())
service = api_instance.read_namespaced_service("istio-ingressgateway", "istio-system", exact='true')
cluster_ip = service.status.load_balancer.ingress[0].ip

In [41]:
url = "http://" + cluster_ip + "/v1/models/cifar10:predict"
headers = { 'Host': 'cifar10-predict.kubeflow.example.com' }
with open('./input.json') as json_file:
    data = json.load(json_file)
    response = requests.post(url, json.dumps(data), headers=headers)
    probs = json.loads(response.content.decode('utf-8'))["predictions"]
    print(probs)
    print(np.argmax(probs))

[[-1.6099601984024048, -2.6461076736450195, 0.32844462990760803, 2.4825074672698975, 0.43524616956710815, 2.3108043670654297, 1.00056791305542, -0.4232763648033142, -0.5100948214530945, -1.7978394031524658]]
3


## Delete the InferenceService

In [45]:
KFServing.delete('cifar10', namespace='kubeflow')

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'cifar10',
  'group': 'serving.kubeflow.org',
  'kind': 'inferenceservices',
  'uid': '7b72288f-de17-11e9-adc5-42010a80027c'}}